In [1]:
import os
import pandas as pd
import email

In [17]:
%%time
# read all files
directory=r'C:\Users\p_uli\Desktop\Columbia University\Cursos\Fall 22\Capstone\Data\maildir'

subset=os.listdir(directory)

msg_id= list()
msg_txt= list()

for folder in subset:
    for roots, dirs, files in os.walk(directory+"\\"+folder):
        for file in files:
            with open(roots+'\\'+file) as f:
                x= f.readlines()
                x=''.join(x)
                msg_txt.append(x)
                msg_id.append(roots[roots.index(folder):] + '\\'+file)


CPU times: total: 47.8 s
Wall time: 40min 11s


In [2]:
data=pd.DataFrame(zip(msg_id,msg_txt),columns=['message_id','message_text'])
data.to_csv(r'C:\Users\p_uli\Desktop\Columbia University\Cursos\Fall 22\Capstone\Data\emails.csv',index=False)


In [3]:
## Helper functions
def get_text_from_email(msg):
    '''To get the content from email objects'''
    parts = []
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            parts.append( part.get_payload() )
    return ''.join(parts)

def split_email_addresses(line):
    '''To separate multiple email addresses'''
    if line:
        addrs = line.split(',')
        addrs = list(map(lambda x: x.strip(), addrs))
    else:
        addrs = None
    return addrs

In [4]:
%%time
# Parse the emails into a list email objects
messages = list(map(email.message_from_string, data['message_text']))
data.drop(columns=['message_text'], inplace=True)

# Get fields from parsed email objects
#keys = messages[0].keys()
keys=['Message-ID', 'Date', 'From', 'To', 'Subject', 'Cc', 'Bcc','X-From', 'X-To', 'X-cc', 'X-bcc', 'X-Folder', 'X-Origin', 'X-FileName']

for key in keys:
    data[key] = [doc[key] for doc in messages]
    
# Parse content from emails
data['body'] = list(map(get_text_from_email, messages))
# Split multiple email addresses
data['From'] = data['From'].map(split_email_addresses)
data['To']   = data['To'].map(split_email_addresses)

# Extract the root of 'file' as 'user'
data['user'] = data['message_id'].map(lambda x:x.split('/')[0])
del messages

data.head(1)

CPU times: total: 1min 11s
Wall time: 1min 44s


,message_id,Message-ID,Date,From,To,Subject,Cc,Bcc,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,body,user
0,allen-p\all_documents\1,<29790972.1075855665306.JavaMail.evans@thyme>,"Wed, 13 Dec 2000 18:41:00 -0800 (PST)",[1.11913372.-2@multexinvestornetwork.com],[pallen@enron.com],"December 14, 2000 - Bear Stearns' predictions ...",None,None,Multex Investor <1.11913372.-2@multexinvestorn...,<pallen@enron.com>,,,\Phillip_Allen_Dec2000\Notes Folders\All docum...,Allen-P,pallen.nsf,In today's Daily Update you'll find free repor...,allen-p\all_documents\1


In [5]:
df=data[['message_id','Date','From','To','Subject','Cc','Bcc','body']]
df=df.rename(columns={"Date": "date", "From": "from",'To':'to','Subject':'subject','Cc':'cc','Bcc':'bcc'})
df.head()

,message_id,date,from,to,subject,cc,bcc,body
0,allen-p\all_documents\1,"Wed, 13 Dec 2000 18:41:00 -0800 (PST)",[1.11913372.-2@multexinvestornetwork.com],[pallen@enron.com],"December 14, 2000 - Bear Stearns' predictions ...",None,None,In today's Daily Update you'll find free repor...
1,allen-p\all_documents\10,"Wed, 13 Dec 2000 08:35:00 -0800 (PST)",[messenger@ecm.bloomberg.com],None,Bloomberg Power Lines Report,None,None,Here is today's copy of Bloomberg Power Lines....
2,allen-p\all_documents\100,"Mon, 9 Oct 2000 07:16:00 -0700 (PDT)",[phillip.allen@enron.com],[keith.holst@enron.com],Consolidated positions: Issues & To Do list,None,None,---------------------- Forwarded by Phillip K ...
3,allen-p\all_documents\101,"Mon, 9 Oct 2000 07:00:00 -0700 (PDT)",[phillip.allen@enron.com],[keith.holst@enron.com],Consolidated positions: Issues & To Do list,None,None,---------------------- Forwarded by Phillip K ...
4,allen-p\all_documents\102,"Thu, 5 Oct 2000 06:26:00 -0700 (PDT)",[phillip.allen@enron.com],[david.delainey@enron.com],,None,None,"Dave, \n\n Here are the names of the west desk..."


In [38]:
df.to_csv(r'C:\Users\p_uli\Desktop\Columbia University\Cursos\Fall 22\Capstone\Data\complete data.csv',index=False)


In [40]:
# Removing duplicates
def tostring(x):
    try:
        return ''.join(x)
    except:
        return 
df['to']=df['to'].apply(lambda x: tostring(x))
df['from']=df['from'].apply(lambda x: tostring(x))

In [41]:
aux=df.drop_duplicates(['date','to','from','subject','cc','bcc','body'])

In [44]:
aux.to_csv(r'C:\Users\p_uli\Desktop\Columbia University\Cursos\Fall 22\Capstone\Data\data_uniques.csv',index=False)

In [12]:
# Number of observations
print(aux.shape)
print(df.shape)

(254912, 8)
(517401, 8)
